In [69]:
import pandas as pd
import tensorflow as tf
import sklearn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
#from glob import glob
from sklearn.utils import shuffle
from PIL import Image

Preprocess Data

In [70]:
#get the shape of one of the images
image_path = r"C:\Users\user\Anaconda\Programs\image_data\train\normal\KEMP_IMG_DATA_1.png"
img = Image.open(image_path)
img_width, img_height = img.size
img_channels = 3 if img.mode == 'RGB' else 1
print("Image Dimensions (Width, Height, Channels):", img_width, img_height, img_channels)

Image Dimensions (Width, Height, Channels): 480 720 1


Image Dataset Creation

In [71]:
class ImageClassification:
    def __init__(self,path1): #path1 represents the directory to the image dataset
        #creating dataframes (df) for the train and test sets
        self.train_df = self._create_df(path1, "train")
        self.test_df = self._create_df(path1, "test")

    #independent utilitity method
    @staticmethod
    #create dataframe with image path and the outcome (either normal or error)
    #construct the dataframes based on the paths to normal and error images found in the directory in path2
    def _create_df(path1, path2):
        #glob.glob is usedd to find all PNG files in the directories
        #files1 = glob.glob(os.path.join(path1+path2, "normal/*.png"))
        #files2 = glob.glob(os.path.join(path1+ path2, "error/*.png"))
        normal_path = os.path.join(path1, path2, "normal", "*png")
        error_path = os.path.join(path1, path2, "error", "*png")
        files1 = glob.glob(normal_path)
        files2 = glob.glob(error_path)
        print("Normal Path:", normal_path)
        print("Error Path:", error_path)
        #prints the number of normal and error images found
        print("normal: ", len(files1))
        print("error: ", len(files2))
            
        #constructs two data frames, df_n = error, df_p = normal
        df_n = pd.DataFrame()
        df_p = pd.DataFrame()
        df_n["name"] = [x for x in files2]
        df_n["outcome"] = 0.0
        df_p["name"] = [x for x in files1]
        df_p["outcome"] = 1.0
        #concatenates and shuffles the dataframes into a single dataframe 'df'
        df = pd.concat([df_n, df_p], axis=0, ignore_index=True)
        df = shuffle(df)
        return df

    #seperate X and Y to train your model
    #operates an instance of class self
    def create_x_and_y(self):
        #need to define preprocessing
        #calls preprocessing method for each image path in train_df
        #creates an array 'X' of preprocessed images
        X = np.array([img_preprocess(p) for p in self.train_df.name.values])
        #retrieve the outcome column values from train_df
        Y = self.train_df.outcome.values
        return X, Y

    @staticmethod
    def img_preprocess(image_path, target_size=(480, 720)):
        #load image
        img = Image.open(image_path)
        #resize image
        img = img.resize(target_size)
        #convert to numpy array and reshape to 480, 720, 1
        img_array = np.array(img).reshape(target_size[0], target_size[1], 1)
        #convert to float32 and normalise value to be between 0 and 1
        img_array = img_array.astype('float32') / 255.0
        return img_array

#get current path of file
dir_path = os.path.join(os.getcwd(), "image_data")
#dir_path = r"C:\Users\user\Anaconda\Programs\Swinburne\archive\image_data"
print("Director Path :", dir_path)
aic = ImageClassification(path1=dir_path)

X, Y = aic.create_x_and_y()

Director Path : C:\Users\user\Anaconda\Programs\image_data
Normal Path: C:\Users\user\Anaconda\Programs\image_data\train\normal\*png
Error Path: C:\Users\user\Anaconda\Programs\image_data\train\error\*png
normal:  1102
error:  59
Normal Path: C:\Users\user\Anaconda\Programs\image_data\test\normal\*png
Error Path: C:\Users\user\Anaconda\Programs\image_data\test\error\*png
normal:  276
error:  15


UnboundLocalError: local variable 'train_df' referenced before assignment